<a href="https://colab.research.google.com/github/arimitramaiti/notebooks/blob/master/Group3_Assignment1_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <center>ARM Basic Implementation</center>
<center>Machine Learning with Big Data (Week-1)</center>
<center>ePGD ABA 2020-21</center>

**Group-3 includes : Nitin Raheja, Rohan Singh, Anand Dattani and Arimitra Maiti**

You are given an online grocery retail dataset. The dataset primarily contains the following transactional attributes: (a) User identifier (b) Order identifier, (c) Product identifier, (d) Product name, (e) Aisle, or location where the product is placed, (f) Department, or category of the product. You are expected to apply rule mining methods on the given dataset, explore interesting patterns and answer the following queries:

In [ ]:
!pip install --quiet mlxtend
!pip install mlxtend --upgrade --no-deps

     |████████████████████████████████| 1.4MB 5.8MB/s 
  Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
pd.set_option('max_colwidth', 300)

In [ ]:
# data = pd.read_excel("E:\\IIM-A Docs\\ML\\Online Grocery Retail Customer Data.xlsx")

In [ ]:
##Download dataset from shared source
url = "https://raw.githubusercontent.com/arimitramaiti/datasets/master/articles/Online%20Grocery%20Retail%20Customer%20Data.csv"
data = pd.read_csv(url, error_bad_lines=False, header=0, index_col=None)

__1) Identify at least six interesting rules from the given dataset. Explain how you determined minimum threshold levels for rule mining__

_All the 6 rules below have high confidence, lift and conviction measures at the same time, all rules are well above minimum support level_

In [ ]:
#additional buy column is created for pivot operation
data['buy'] = 1
print(data.shape)
dfproc = data.pivot_table(index=["oid"], columns="pname", values='buy', aggfunc="count", fill_value=0)
dfproc

#set the parameter values appropriately, random values are set below
#max_len field is set to restrict the size of itemsets. If you wish to get all itemsets do not specify the parameter
#use_colnames = True gives the actual column names. By default (False), it displays item identifiers

frequent_items = fpgrowth(dfproc, min_support=0.02, use_colnames=True) #use this by default
frequent_items['size'] = frequent_items["itemsets"].apply(lambda x: len(x))
frequent_items.head(10) #display 10 results

#specify suitable parameter values, random values are set below
rules = association_rules(frequent_items, metric="support", min_threshold=0.02)
rules

#filter rules based on measures
rules[ (rules['confidence'] > 0.53) & (rules['lift'] > 1) ].sort_values(by=['confidence', 'lift'], ascending=[False, False])

(75005, 9)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
205,(Icelandic Style Skyr Blueberry Non-fat Yogurt),(Nonfat Icelandic Style Strawberry Yogurt),0.032239,0.034877,0.026671,0.827273,23.719786,0.025546,5.587555
204,(Nonfat Icelandic Style Strawberry Yogurt),(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.034877,0.032239,0.026671,0.764706,23.719786,0.025546,4.112984
26,"(Bag of Organic Bananas, Organic Whole String Cheese)",(Organic Strawberries),0.034291,0.250293,0.021102,0.615385,2.458656,0.012519,1.949238
167,(Organic Garnet Sweet Potato (Yam)),(Bag of Organic Bananas),0.048652,0.248828,0.028722,0.590361,2.372572,0.016616,1.833744
17,(Organic Whole String Cheese),(Organic Strawberries),0.080012,0.250293,0.045721,0.571429,2.283038,0.025695,1.749316
88,"(Organic Hass Avocado, Organic Raspberries)",(Bag of Organic Bananas),0.041618,0.248828,0.022567,0.542254,2.179233,0.012212,1.641023


__2) What product bundles would you consider for boosting sales of ‘Moroccan Mint Green Tea’?__

_Bundling the following consequents with high consequent support as well as itemset correlations suggests they can boost sales of Green tea in lieu of increased bundle value, keeping in mind binary partition of a rule as A implies B also suggests B implies A_

In [ ]:
rules['antecedents']="Moroccan Mint Green Tea"
#filter rules based on measures
rules[ (rules['consequent support'] > 0.20) & (rules['lift'] > 1) & (rules['antecedent support'] > 0.20) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Moroccan Mint Green Tea,(Organic Strawberries),0.248828,0.250293,0.074443,0.299176,1.195301,0.012163,1.069750
1,Moroccan Mint Green Tea,(Bag of Organic Bananas),0.250293,0.248828,0.074443,0.297424,1.195301,0.012163,1.069169
104,Moroccan Mint Green Tea,(Organic Strawberries),0.213658,0.250293,0.082943,0.388203,1.550994,0.029466,1.225418
105,Moroccan Mint Green Tea,(Organic Hass Avocado),0.250293,0.213658,0.082943,0.331382,1.550994,0.029466,1.176071
106,Moroccan Mint Green Tea,(Organic Hass Avocado),0.248828,0.213658,0.085580,0.343934,1.609743,0.032416,1.198572
107,Moroccan Mint Green Tea,(Bag of Organic Bananas),0.213658,0.248828,0.085580,0.400549,1.609743,0.032416,1.253100


__3) How would you leverage the aisle and department level information in your analysis? Prepare rule mining workflows using aisle and department level information to demonstrate your ideas.__

__Aisle level__

_From local implementation above we evaluated the density at aisle level is 10% which is higher than the product level density at 0.03%. Therefore aisle is at higher level than products is justified here. Now we need to set a minimum threshold over and above 10% density as our minimum support threshold value. We set this is 20% to 30% for aisle level_

In [ ]:
##At aisle level which is one level higher than product name
#additional buy column is created for pivot operation
aisle = data.pivot_table(index=["oid"], columns="aisle", values='buy', aggfunc="count",
                                          fill_value=0)

#By definition we know each rule is a binary partition,
#So any cell containing a value more than 1 from pivot implies transaction is made and reflects quantity
#To translate the basic binary transaction detection all cell values of sparse matrix greater than equal to 1 must contain 1
#All cell values cell values of sparse matrix with zero must imply transaction has not taken place
for i in aisle.columns:
    aisle[i] = aisle[i].apply(lambda x: 1 if x >= 1 else 0)

##evaluate density to check how large can minimum support threshold be
density_aisle = round((aisle.iloc[:, :].sum(axis=0).sum()/(aisle.shape[0]*aisle.shape[1])),2)
density_aisle

0.1

In [ ]:
frequent_items_aisle = fpgrowth(aisle, min_support=0.20, use_colnames=True) #use this by default
frequent_items_aisle.head(10) #display 10 results

#specify suitable parameter values, random values are set below
rules_aisle = association_rules(frequent_items_aisle, metric="support", min_threshold=0.30)
rules_aisle = rules_aisle.sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

#filter rules based on measures
rules_aisle[ (rules_aisle['confidence'] > 0.90) & (rules_aisle['lift'] > 1) & (rules_aisle['conviction'] > 2) ].sort_values(by=['confidence', 'lift', 'conviction'], ascending=[False, False, False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
46,"(fresh vegetables, milk)",(fresh fruits),0.338804,0.830598,0.316823,0.935121,1.125841,0.035413,2.611051
52,"(milk, packaged vegetables fruits)",(fresh fruits),0.338218,0.830598,0.314771,0.930676,1.120489,0.033848,2.443625
28,"(fresh vegetables, packaged cheese)",(fresh fruits),0.366061,0.830598,0.340563,0.930344,1.120090,0.036513,2.431991
34,"(packaged cheese, packaged vegetables fruits)",(fresh fruits),0.354631,0.830598,0.329132,0.928099,1.117387,0.034577,2.356052
10,"(fresh vegetables, yogurt)",(fresh fruits),0.374853,0.830598,0.347890,0.928069,1.117350,0.036537,2.355058
68,"(fresh vegetables, packaged vegetables fruits)",(fresh fruits),0.504396,0.830598,0.466589,0.925044,1.113708,0.047638,2.260008


__Department level__

_From local implementation above we evaluated the density at department level is 35% which is higher than the product level density at 0.03%. Therefore department is at higher level than aisle or products is justified here. Now we need to set a minimum threshold over and above 35% density as our minimum support threshold value. We set this is 60% for department level_

In [ ]:
##At department level which is one level higher than product name
#additional buy column is created for pivot operation
department = data.pivot_table(index=["oid"], columns="department", values='buy', aggfunc="count",
                                          fill_value=0)

#By definition we know each rule is a binary partition,
#So any cell containing a value more than 1 from pivot implies transaction is made and reflects quantity
#To translate the basic binary transaction detection all cell values of sparse matrix greater than equal to 1 must contain 1
#All cell values cell values of sparse matrix with zero must imply transaction has not taken place
for i in department.columns:
    department[i] = department[i].apply(lambda x: 1 if x >= 1 else 0)

##evaluate density to check how large can minimum support threshold be
density_department = round((department.iloc[:, :].sum(axis=0).sum()/(department.shape[0]*department.shape[1])),2)
density_department

0.35

In [ ]:
frequent_items_department = fpgrowth(department, min_support=0.60, use_colnames=True) #use this by default
frequent_items_department.head(10) #display 10 results

,support,itemsets
0,0.928195,(produce)
1,0.917937,(dairy eggs)
2,0.694607,(snacks)
3,0.668230,(beverages)
4,0.872216,"(produce, dairy eggs)"
5,0.663540,"(snacks, dairy eggs)"
6,0.659437,"(produce, snacks)"
7,0.636870,"(produce, dairy eggs, snacks)"
8,0.630715,"(dairy eggs, beverages)"
9,0.627784,"(produce, beverages)"


In [ ]:
#specify suitable parameter values, random values are set below
rules_department = association_rules(frequent_items_department, metric="support", min_threshold=0.60)
rules_department = rules_department.sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

In [ ]:
rules_department

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(dairy eggs),(produce),0.917937,0.928195,0.872216,0.950192,1.023699,0.020192,1.441631
0,(produce),(dairy eggs),0.928195,0.917937,0.872216,0.939691,1.023699,0.020192,1.360704
2,(snacks),(dairy eggs),0.694607,0.917937,0.663540,0.955274,1.040676,0.025935,1.834812
3,(dairy eggs),(snacks),0.917937,0.694607,0.663540,0.722861,1.040676,0.025935,1.101947
5,(snacks),(produce),0.694607,0.928195,0.659437,0.949367,1.022810,0.014707,1.418157
4,(produce),(snacks),0.928195,0.694607,0.659437,0.710452,1.022810,0.014707,1.054721
7,"(produce, snacks)",(dairy eggs),0.659437,0.917937,0.636870,0.965778,1.052118,0.031548,2.397954
8,"(snacks, dairy eggs)",(produce),0.663540,0.928195,0.636870,0.959806,1.034056,0.020975,1.786455
11,(snacks),"(produce, dairy eggs)",0.694607,0.872216,0.636870,0.916878,1.051205,0.031022,1.537303
6,"(produce, dairy eggs)",(snacks),0.872216,0.694607,0.636870,0.730175,1.051205,0.031022,1.131817


In [ ]:
#filter rules based on measures
rules_department[ (rules_department['confidence'] > 0.90) & (rules_department['lift'] > 1) & (rules_department['conviction'] > 2) ].sort_values(by=['confidence', 'lift', 'conviction'], ascending=[False, False, False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
7,"(produce, snacks)",(dairy eggs),0.659437,0.917937,0.636870,0.965778,1.052118,0.031548,2.397954
17,"(produce, beverages)",(dairy eggs),0.627784,0.917937,0.603751,0.961718,1.047695,0.027485,2.143654


__Combined Product, Aisle and Department levels__

_From local implementation above we evaluated the density at combined level is same as the product level density at 0.03%. Therefore combined level is NOT at higher level than department or aisle or products is justified here. Now we need to set a minimum threshold over and above .03% density as our minimum support threshold value. We set this is 3%, which is slightly higher than 2% tried above while mining product level rules_

In [ ]:
##At product, aisle and department level which is one level higher than product name
#additional buy column is created for pivot operation
pname_aisle_department = data.pivot_table(index=["oid"], columns="pname_aisle_department", values='buy', aggfunc="count",
                                          fill_value=0)

#By definition we know each rule is a binary partition,
#So any cell containing a value more than 1 from pivot implies transaction is made and reflects quantity
#To translate the basic binary transaction detection all cell values of sparse matrix greater than equal to 1 must contain 1
#All cell values cell values of sparse matrix with zero must imply transaction has not taken place
for i in pname_aisle_department.columns:
    pname_aisle_department[i] = pname_aisle_department[i].apply(lambda x: 1 if x >= 1 else 0)

##evaluate density to check how large can minimum support threshold be
density_pname_aisle_department = round((pname_aisle_department.iloc[:, :].sum(axis=0).sum()/(pname_aisle_department.shape[0]*pname_aisle_department.shape[1])),4)
density_pname_aisle_department

0.0031

In [ ]:
frequent_items_combined = fpgrowth(pname_aisle_department, min_support=0.03, use_colnames=True) #use this by default
frequent_items_combined.head(10) #display 10 results

,support,itemsets
0,0.250293,(Organic Strawberries::fresh fruits::produce)
1,0.248828,(Bag of Organic Bananas::fresh fruits::produce)
2,0.089977,(Organic Avocado::fresh fruits::produce)
3,0.086753,(Organic Large Extra Fancy Fuji Apple::fresh fruits::produce)
4,0.080012,(Organic Whole String Cheese::packaged cheese::dairy eggs)
5,0.051290,(Organic Small Bunch Celery::fresh vegetables::produce)
6,0.042497,(Organic Cilantro::fresh herbs::produce)
7,0.033705,(Organic Red Onion::fresh vegetables::produce)
8,0.030188,(Colby Cheese Sticks::packaged cheese::dairy eggs)
9,0.158558,(Organic Baby Spinach::packaged vegetables fruits::produce)


In [ ]:
#specify suitable parameter values, random values are set below
rules_combined = association_rules(frequent_items_combined, metric="support", min_threshold=0.03)
rules_combined = rules_combined.sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

In [ ]:
rules_combined

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,(Organic Hass Avocado::fresh fruits::produce),(Bag of Organic Bananas::fresh fruits::produce),0.213658,0.248828,0.085580,0.400549,1.609743,0.032416,1.253100
36,(Bag of Organic Bananas::fresh fruits::produce),(Organic Hass Avocado::fresh fruits::produce),0.248828,0.213658,0.085580,0.343934,1.609743,0.032416,1.198572
35,(Organic Hass Avocado::fresh fruits::produce),(Organic Strawberries::fresh fruits::produce),0.213658,0.250293,0.082943,0.388203,1.550994,0.029466,1.225418
34,(Organic Strawberries::fresh fruits::produce),(Organic Hass Avocado::fresh fruits::produce),0.250293,0.213658,0.082943,0.331382,1.550994,0.029466,1.176071
1,(Bag of Organic Bananas::fresh fruits::produce),(Organic Strawberries::fresh fruits::produce),0.248828,0.250293,0.074443,0.299176,1.195301,0.012163,1.069750
0,(Organic Strawberries::fresh fruits::produce),(Bag of Organic Bananas::fresh fruits::produce),0.250293,0.248828,0.074443,0.297424,1.195301,0.012163,1.069169
29,(Organic Raspberries::packaged vegetables fruits::produce),(Bag of Organic Bananas::fresh fruits::produce),0.142145,0.248828,0.058910,0.414433,1.665542,0.023540,1.282812
28,(Bag of Organic Bananas::fresh fruits::produce),(Organic Raspberries::packaged vegetables fruits::produce),0.248828,0.142145,0.058910,0.236749,1.665542,0.023540,1.123948
27,(Banana::fresh fruits::produce),(Organic Strawberries::fresh fruits::produce),0.195780,0.250293,0.054513,0.278443,1.112468,0.005511,1.039013
26,(Organic Strawberries::fresh fruits::produce),(Banana::fresh fruits::produce),0.250293,0.195780,0.054513,0.217799,1.112468,0.005511,1.028150


In [ ]:
#filter rules based on measures
rules_combined[ (rules_combined['confidence'] > 0.45) & (rules_combined['lift'] > 1) & (rules_combined['conviction'] > 1) ].sort_values(by=['confidence', 'lift', 'conviction'], ascending=[False, False, False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9,(Organic Whole String Cheese::packaged cheese::dairy eggs),(Organic Strawberries::fresh fruits::produce),0.080012,0.250293,0.045721,0.571429,2.283038,0.025695,1.749316
38,"(Organic Strawberries::fresh fruits::produce, Bag of Organic Bananas::fresh fruits::produce)",(Organic Hass Avocado::fresh fruits::produce),0.074443,0.213658,0.037515,0.503937,2.358619,0.021609,1.585166
40,"(Organic Strawberries::fresh fruits::produce, Organic Hass Avocado::fresh fruits::produce)",(Bag of Organic Bananas::fresh fruits::produce),0.082943,0.248828,0.037515,0.452297,1.817711,0.016876,1.371495


__4) You wish to discontinue ‘Organic Orange Juice’ product. Assess the implications of your decision__

_There is a strong association between {Organic Orange Juice and Organic Hass Avocado} and {Organic Orange Juice and Bag of Organic Bananas}, however we observe that this takes place in roughly 1.14% of transactions. Surprisingly we also observe there is a strong association between {Organic Hass Avocado and Bag of Organic Bananas} where the overall support is most of times higher than 1% minimum support. We may conclude that Organic Orange Juice has a potential to serve as a link between avocado and organic banana bag. Therefore if we disconitnue the organic orange juice then its association with avocado and banana bag would be broken thereby affecting an adverse impact between avocado and banana bag. Therefore we may think of bundling avocado, organic banana bag and organic orange juice together as part of a promotion_

_Discontinuing an item based on its support value alone (which is a little over 1% here) may not be a wise decision to consider the varitey of product promotions possible. Hence creating efficient bundles with avocado and organic banana may apprise orange juice preference_

**For this product level analysis we lower the support from 3% to 1% as our minimum support threshold**

In [ ]:
frequent_items_OOJ = fpgrowth(dfproc, min_support=0.01, use_colnames=True) #use this by default
frequent_items_OOJ['size'] = frequent_items["itemsets"].apply(lambda x: len(x))
frequent_items_OOJ.head(10) #display 10 results

#specify suitable parameter values, random values are set below
rules_OOJ = association_rules(frequent_items_OOJ, metric="support", min_threshold=0.01)

_We do notice Organic Orange Juice has high correlation with avocado and bananas, the high conviction number implies this correlation is not by **chance_**

In [ ]:
df_orangejuiceLHS = rules_OOJ[rules_OOJ["antecedents"].apply(lambda x: 'Organic Orange Juice' in str(x))]
df_orangejuiceLHS

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6908,(Organic Orange Juice),(Organic Hass Avocado),0.016999,0.213658,0.01143,0.672414,3.147155,0.007798,2.400413
6910,(Organic Orange Juice),(Bag of Organic Bananas),0.016999,0.248828,0.01143,0.672414,2.702327,0.007200,2.293052


_We do notice avocado and bananas also have high correlation with Organic Orange Juice , the high conviction number implies this correlation is not by **chance_**

In [ ]:
df_orangejuiceRHS = rules_OOJ[rules_OOJ["consequents"].apply(lambda x: 'Organic Orange Juice' in str(x))]
df_orangejuiceRHS

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6909,(Organic Hass Avocado),(Organic Orange Juice),0.213658,0.016999,0.01143,0.053498,3.147155,0.007798,1.038562
6911,(Bag of Organic Bananas),(Organic Orange Juice),0.248828,0.016999,0.01143,0.045936,2.702327,0.007200,1.030331


In [ ]:
Avocado = rules_OOJ[rules_OOJ["antecedents"].apply(lambda x: 'Organic Hass Avocado' in str(x))].sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

Avocado = Avocado[Avocado["consequents"].apply(lambda x: 'Bag of Organic Bananas' in str(x))].sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))
Avocado.shape

(47, 9)

_Relation between Organic avocado and organic bag of bananas_

In [ ]:
Avocado.sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5027,(Organic Hass Avocado),(Bag of Organic Bananas),0.213658,0.248828,0.085580,0.400549,1.609743,0.032416,1.253100
5030,"(Organic Hass Avocado, Organic Strawberries)",(Bag of Organic Bananas),0.082943,0.248828,0.037515,0.452297,1.817711,0.016876,1.371495
5033,(Organic Hass Avocado),"(Bag of Organic Bananas, Organic Strawberries)",0.213658,0.074443,0.037515,0.175583,2.358619,0.021609,1.122680
4904,"(Organic Hass Avocado, Organic Raspberries)",(Bag of Organic Bananas),0.041618,0.248828,0.022567,0.542254,2.179233,0.012212,1.641023
4906,(Organic Hass Avocado),"(Bag of Organic Bananas, Organic Raspberries)",0.213658,0.058910,0.022567,0.105624,1.792983,0.009981,1.052231
222,"(Organic Hass Avocado, Organic Baby Spinach)",(Bag of Organic Bananas),0.047186,0.248828,0.019343,0.409938,1.647477,0.007602,1.273039
224,(Organic Hass Avocado),"(Bag of Organic Bananas, Organic Baby Spinach)",0.213658,0.044842,0.019343,0.090535,2.018989,0.009763,1.050242
67,"(Organic Large Extra Fancy Fuji Apple, Organic Hass Avocado)",(Bag of Organic Bananas),0.027550,0.248828,0.018757,0.680851,2.736235,0.011902,2.353673
71,(Organic Hass Avocado),"(Organic Large Extra Fancy Fuji Apple, Bag of Organic Bananas)",0.213658,0.035463,0.018757,0.087791,2.475575,0.011180,1.057365
5071,"(Organic Whole Milk, Organic Hass Avocado)",(Bag of Organic Bananas),0.038980,0.248828,0.016999,0.436090,1.752579,0.007300,1.332079


In [ ]:
OB = rules_OOJ[rules_OOJ["antecedents"].apply(lambda x: 'Bag of Organic Bananas' in str(x))].sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

OB = OB[OB["consequents"].apply(lambda x: 'Organic Hass Avocado' in str(x))].sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))
OB.shape

(47, 9)

_Relation between Organic bananas and organic avocado_

In [ ]:
OB.sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5026,(Bag of Organic Bananas),(Organic Hass Avocado),0.248828,0.213658,0.085580,0.343934,1.609743,0.032416,1.198572
5028,"(Bag of Organic Bananas, Organic Strawberries)",(Organic Hass Avocado),0.074443,0.213658,0.037515,0.503937,2.358619,0.021609,1.585166
5031,(Bag of Organic Bananas),"(Organic Hass Avocado, Organic Strawberries)",0.248828,0.082943,0.037515,0.150766,1.817711,0.016876,1.079864
4903,"(Bag of Organic Bananas, Organic Raspberries)",(Organic Hass Avocado),0.058910,0.213658,0.022567,0.383085,1.792983,0.009981,1.274636
4905,(Bag of Organic Bananas),"(Organic Hass Avocado, Organic Raspberries)",0.248828,0.041618,0.022567,0.090695,2.179233,0.012212,1.053972
221,"(Bag of Organic Bananas, Organic Baby Spinach)",(Organic Hass Avocado),0.044842,0.213658,0.019343,0.431373,2.018989,0.009763,1.382878
223,(Bag of Organic Bananas),"(Organic Hass Avocado, Organic Baby Spinach)",0.248828,0.047186,0.019343,0.077739,1.647477,0.007602,1.033127
66,"(Organic Large Extra Fancy Fuji Apple, Bag of Organic Bananas)",(Organic Hass Avocado),0.035463,0.213658,0.018757,0.528926,2.475575,0.011180,1.669253
70,(Bag of Organic Bananas),"(Organic Large Extra Fancy Fuji Apple, Organic Hass Avocado)",0.248828,0.027550,0.018757,0.075383,2.736235,0.011902,1.051733
5070,"(Organic Whole Milk, Bag of Organic Bananas)",(Organic Hass Avocado),0.029015,0.213658,0.016999,0.585859,2.742043,0.010799,1.898729


__5) Suppose that the following rules are generated by mining the transaction histories at the aisle level (at suitable minimum threshold levels).__

Soft drinks => Fresh fruits;
Candy chocolate => Fresh vegetables;
Tea => Fresh fruits;
Soft drinks => Fresh vegetables;
Coffee => Fresh fruits

__Explain how you would use the above rules for making promotional decisions. What rule measures did you consider while making your decision?__

_Complement the consequent of dataframe-c with reference to the consequent of dataframe-b, i.e. pick items with high complimentarity from dataframe-b and subsititute them (or create promotional bundles) with the consequents of dataframe-c_

**Earlier we had set 30% minimum support at aisle level, however to cover soft drinks, tea and coffee we have to lower the same to 5%**

In [ ]:
frequent_items_aisle = fpgrowth(aisle, min_support=0.05, use_colnames=True) #use this by default
frequent_items_aisle.head(10) #display 10 results

#specify suitable parameter values, random values are set below
rules_aisle = association_rules(frequent_items_aisle, metric="support", min_threshold=0.05)
rules_aisle = rules_aisle.sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

In [ ]:
##Convert frozenset string to unicode string
rules_aisle["antecedents"] = rules_aisle["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
rules_aisle["consequents"] = rules_aisle["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

In [ ]:
rules_aisle.shape

(36244, 9)

In [ ]:
##Store the distinct items mentioned in the rules above
items = ['soft drinks', 'fresh fruits', 'candy chocolate', 'fresh vegetables', 'tea', 'coffee']
##Search the list to extract all rules where these items exist
rules_aisle['LHS'] = rules_aisle['antecedents'].str.findall('(' + '|'.join(items) + ')')
rules_aisle['RHS'] = rules_aisle['consequents'].str.findall('(' + '|'.join(items) + ')')

In [ ]:
rules_aisle.shape

(36244, 11)

In [ ]:
##Remove blank rows where the items dont exist
aislepromo = rules_aisle[rules_aisle['LHS'].astype(bool) & rules_aisle['RHS'].astype(bool)]
aislepromo = aislepromo.iloc[:, :9].sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))
##Check the dimensions of the dataset
aislepromo.shape

(6990, 9)

__Soft Drinks__

In [ ]:
a = aislepromo[(aislepromo['lift']> 1) & (aislepromo['antecedents'].str.contains('soft drinks')) &
               (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"],
                                                             ascending = (False, False, False, False))

In [ ]:
##Implies complimentary bundles with high confidence
b = a[a['lift']==a['lift'].max()]
b

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
34977,"soft drinks, fresh vegetables","fresh fruits, packaged vegetables fruits",0.078839,0.590563,0.05891,0.747212,1.265254,0.01235,1.619686


In [ ]:
##Implies substitutability bundles with high confidence
c = aislepromo[(aislepromo['lift']< 1) & (aislepromo['lift']>aislepromo['lift'].min()) &
               (aislepromo['antecedents'].str.contains('soft drinks')) &
               (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"],
                                                             ascending = (False, False, False, False))
c

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
34943,soft drinks,fresh fruits,0.118699,0.830598,0.097011,0.817284,0.983971,-0.001580,0.927133
34936,soft drinks,fresh vegetables,0.118699,0.669402,0.078839,0.664198,0.992225,-0.000618,0.984501
34968,soft drinks,"fresh fruits, fresh vegetables",0.118699,0.606389,0.070633,0.595062,0.981320,-0.001345,0.972027


__Fresh Fruits__

In [ ]:
a = aislepromo[(aislepromo['lift']> 1) & (aislepromo['antecedents'].str.contains('fresh fruits')) &
               (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"],
                                                             ascending = (False, False, False, False))

In [ ]:
##Implies complimentary bundles with high confidence
b = a[a['lift']==a['lift'].max()]
b

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9967,"fresh fruits, tortillas flat bread","fresh vegetables, bread",0.104045,0.251172,0.052462,0.504225,2.007488,0.026329,1.510419


In [ ]:
##Implies substitutability bundles with high confidence
c = aislepromo[(aislepromo['lift']< 1) & (aislepromo['lift']>aislepromo['lift'].min()) &
               (aislepromo['antecedents'].str.contains('fresh fruits')) &
               (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"],
                                                             ascending = (False, False, False, False))
c

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23317,"fresh fruits, coffee",fresh vegetables,0.092321,0.669402,0.056565,0.612698,0.915292,-0.005235,0.853593


__Candy Chocolate__

In [ ]:
a = aislepromo[(aislepromo['lift']> 1) & (aislepromo['antecedents'].str.contains('candy chocolate')) &
               (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"],
                                                             ascending = (False, False, False, False))

In [ ]:
##Implies complimentary bundles with high confidence
b = a[a['lift']==a['lift'].max()]
b

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23493,"candy chocolate, fresh vegetables","fresh fruits, packaged vegetables fruits",0.075029,0.590563,0.055393,0.738281,1.250132,0.011083,1.564417


In [ ]:
##Implies substitutability bundles with high confidence
c = aislepromo[(aislepromo['lift']< 1) & (aislepromo['lift']>aislepromo['lift'].min()) &
               (aislepromo['antecedents'].str.contains('candy chocolate')) &
               (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"],
                                                             ascending = (False, False, False, False))
c

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23459,candy chocolate,fresh vegetables,0.113423,0.669402,0.075029,0.661499,0.988193,-0.000896,0.976652


__Fresh Vegetables__

In [ ]:
a = aislepromo[(aislepromo['lift']> 1) & (aislepromo['antecedents'].str.contains('fresh vegetables')) &
               (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"],
                                                             ascending = (False, False, False, False))

In [ ]:
##Implies complimentary bundles with high confidence
b = a[a['lift']==a['lift'].max()]
b

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
13008,"fresh vegetables, lunch meat, yogurt","fresh fruits, packaged cheese, packaged vegetables fruits",0.109027,0.329132,0.068581,0.629032,1.911183,0.032697,1.808426


In [ ]:
##Implies substitutability bundles with high confidence
c = aislepromo[(aislepromo['lift']< 1) & (aislepromo['lift']>aislepromo['lift'].min()) &
               (aislepromo['antecedents'].str.contains('fresh vegetables')) &
               (aislepromo['confidence']> 0.40)].sort_values(["support", "confidence", "lift", "conviction"],
                                                             ascending = (False, False, False, False))
c

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2360,"fresh vegetables, soy lactosefree","fresh fruits, milk",0.274033,0.432298,0.113423,0.413904,0.957451,-0.005041,0.968616
2393,"fresh vegetables, soy lactosefree, packaged vegetables fruits","fresh fruits, milk",0.225381,0.432298,0.094666,0.420026,0.971613,-0.002766,0.978841
17578,"fresh vegetables, soup broth bouillon","fresh fruits, milk",0.125733,0.432298,0.050703,0.403263,0.932837,-0.003651,0.951345


__Tea__

In [ ]:
a = aislepromo[(aislepromo['lift']> 1) & (aislepromo['antecedents'].str.contains('tea')) &
               (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"],
                                                             ascending = (False, False, False, False))

In [ ]:
##Implies complimentary bundles with high confidence
b = a[a['lift']==a['lift'].max()]
b

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
17967,"tea, fresh vegetables","fresh fruits, packaged vegetables fruits",0.081184,0.590563,0.061841,0.761733,1.289842,0.013896,1.718396


In [ ]:
##Implies substitutability bundles with high confidence
c = aislepromo[(aislepromo['lift']< 1) & (aislepromo['lift']>aislepromo['lift'].min()) &
               (aislepromo['antecedents'].str.contains('tea')) &
               (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"],
                                                             ascending = (False, False, False, False))
c

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
17919,tea,fresh fruits,0.128957,0.830598,0.106096,0.822727,0.990524,-0.001015,0.955602
17922,tea,fresh vegetables,0.128957,0.669402,0.081184,0.629545,0.940459,-0.005140,0.892411
17952,tea,"fresh fruits, fresh vegetables",0.128957,0.606389,0.072685,0.563636,0.929496,-0.005513,0.902025
18000,"tea, yogurt",fresh fruits,0.063306,0.830598,0.052462,0.828704,0.997719,-0.000120,0.988942


__Coffee__

In [ ]:
a = aislepromo[(aislepromo['lift']> 1) & (aislepromo['antecedents'].str.contains('coffee')) & (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

In [ ]:
##Implies complimentary bundles with high confidence
b = a[a['lift']==a['lift'].max()]
b

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23318,"fresh vegetables, coffee",fresh fruits,0.061547,0.830598,0.056565,0.919048,1.106489,0.005444,2.092614


In [ ]:
##Implies substitutability bundles with high confidence
c = aislepromo[(aislepromo['lift']< 1) & (aislepromo['lift']>aislepromo['lift'].min()) & (aislepromo['antecedents'].str.contains('coffee')) & (aislepromo['confidence']> 0.50)].sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))
c

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23283,coffee,fresh fruits,0.111958,0.830598,0.092321,0.824607,0.992788,-0.000671,0.965845
23317,"fresh fruits, coffee",fresh vegetables,0.092321,0.669402,0.056565,0.612698,0.915292,-0.005235,0.853593
23321,coffee,"fresh fruits, fresh vegetables",0.111958,0.606389,0.056565,0.505236,0.833187,-0.011325,0.795552
23324,"coffee, packaged vegetables fruits",fresh vegetables,0.077081,0.669402,0.050703,0.657795,0.982660,-0.000895,0.966081


__How would you generate promotional offers for a given customer, e.g. uid = 4962?__

- _Bundle milk, coffee, lunch meat, refrigerated items in separate bundles with fresh items for breakfast and lunch separately (insights from aisle level)_
- _Create a specialised bundle of all kinds of organic fruits that would include avocado and strawberry (insights from combined level)_

In [ ]:
data[data['uid']==4962].department.unique()
##Not all departments of this customer fall under minimum support of 60% for above analyzed aisle rules.
## therefore either we need to reduce the department minimum support or we need to go further level down
##either check aisle levels or product levels

array(['breakfast', 'dairy eggs', 'produce', 'meat seafood', 'bakery',
       'snacks', 'frozen', 'deli', 'beverages', 'pantry', 'personal care',
       'household', 'other'], dtype=object)

__Aisle level for Customer 4962__

In [ ]:
items_4962 = list(data[data['uid']==4962].aisle.unique())
items_4962 ##44 unique items purchased so far

['granola',
 'soy lactosefree',
 'packaged vegetables fruits',
 'packaged cheese',
 'fresh vegetables',
 'yogurt',
 'packaged poultry',
 'eggs',
 'fresh fruits',
 'bread',
 'candy chocolate',
 'frozen dessert',
 'frozen vegan vegetarian',
 'prepared soups salads',
 'frozen meals',
 'butter',
 'coffee',
 'refrigerated',
 'frozen breads doughs',
 'chips pretzels',
 'hot dogs bacon sausage',
 'baking ingredients',
 'milk',
 'oral hygiene',
 'juice nectars',
 'tea',
 'frozen breakfast',
 'lunch meat',
 'spreads',
 'oils vinegars',
 'air fresheners candles',
 'spices seasonings',
 'frozen produce',
 'vitamins supplements',
 'cleaning products',
 'dish detergents',
 'paper goods',
 'fresh herbs',
 'other creams cheeses',
 'fresh dips tapenades',
 'honeys syrups nectars',
 'other',
 'digestion',
 'hot cereal pancake mixes']

In [ ]:
frequent_items_aisle = fpgrowth(aisle, min_support=0.05, use_colnames=True) #use this by default
frequent_items_aisle.head(10) #display 10 results

#specify suitable parameter values, random values are set below
rules_aisle = association_rules(frequent_items_aisle, metric="support", min_threshold=0.05)
rules_aisle = rules_aisle.sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

In [ ]:
##Convert frozenset string to unicode string
rules_aisle["antecedents"] = rules_aisle["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
rules_aisle["consequents"] = rules_aisle["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

##Find all aisles visited by customer id 4962
rules_aisle['LHS'] = rules_aisle['antecedents'].str.findall('(' + '|'.join(items_4962) + ')')
rules_aisle['RHS'] = rules_aisle['consequents'].str.findall('(' + '|'.join(items_4962) + ')')

##Remove blank rows where the items dont exist
custpromo = rules_aisle[rules_aisle['LHS'].astype(bool) & rules_aisle['RHS'].astype(bool)]
custpromo = custpromo.iloc[:, :9].sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

custpromo.shape

(33614, 9)

In [ ]:
custpromo['lift'].describe()

count    33614.000000
mean         1.324730
std          0.179043
min          0.821238
25%          1.186631
50%          1.300495
75%          1.438553
max          2.122159
Name: lift, dtype: float64

In [ ]:
custpromo['conviction'].describe()

count    33614.000000
mean         1.277953
std          0.543689
min          0.734235
25%          1.044190
50%          1.114838
75%          1.297460
max         15.754396
Name: conviction, dtype: float64

_Complimentary rules suggest bread & fresh vegetables go extremely well with fresh fruits and flat bread_

In [ ]:
custpromo[(custpromo['lift']>2) & (custpromo['confidence']>.50)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9967,"fresh fruits, tortillas flat bread","fresh vegetables, bread",0.104045,0.251172,0.052462,0.504225,2.007488,0.026329,1.510419


_Packaged vegetables, fruits, chips go well with fresh fruits and lactose free product and lunch meat, so customer buys lunch meat with an assortment mostly_

In [ ]:
custpromo[(custpromo['lift']<2) & (custpromo['lift']>1.95) & (custpromo['confidence']>.50)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10826,"fresh fruits, lunch meat, soy lactosefree","chips pretzels, packaged vegetables fruits",0.100528,0.267585,0.053341,0.530612,1.982967,0.026441,1.560362


_Relatively less complimentary rules suggest customer prefers fresh items, he/she prefres yogurt, chips, milk most of the times with fresh fruits or vegetables_

In [ ]:
custpromo[(custpromo['lift']>1) & (custpromo['lift']<1.10) & (custpromo['confidence']>.50)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,fresh vegetables,fresh fruits,0.669402,0.830598,0.606389,0.905867,1.090620,0.050385,1.799602
0,fresh fruits,fresh vegetables,0.830598,0.669402,0.606389,0.730064,1.090620,0.050385,1.224725
10187,packaged vegetables fruits,fresh fruits,0.660023,0.830598,0.590563,0.894760,1.077248,0.042349,1.609677
10186,fresh fruits,packaged vegetables fruits,0.830598,0.660023,0.590563,0.711009,1.077248,0.042349,1.176427
3,yogurt,fresh fruits,0.516999,0.830598,0.450762,0.871882,1.049704,0.021344,1.322236
...,...,...,...,...,...,...,...,...,...
33084,"water seltzer sparkling water, milk, chips pretzels",fresh vegetables,0.073857,0.669402,0.050117,0.678571,1.013698,0.000677,1.028527
31068,"spreads, chips pretzels","fresh fruits, fresh vegetables",0.079426,0.606389,0.050117,0.630996,1.040580,0.001954,1.066685
17511,"fresh fruits, fresh vegetables, soup broth bouillon, packaged vegetables fruits",yogurt,0.094373,0.516999,0.050117,0.531056,1.027190,0.001327,1.029976
5674,"bread, fresh vegetables, chips pretzels, packaged vegetables fruits",milk,0.095545,0.485639,0.050117,0.524540,1.080103,0.003717,1.081817


_Substitue rules suggest customer purchase cofee only with fresh items again, with high confidence_

In [ ]:
custpromo[(custpromo['lift']<.90) & (custpromo['confidence']>.50)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23291,coffee,fresh vegetables,0.111958,0.669402,0.061547,0.549738,0.821238,-0.013397,0.734235
23321,coffee,"fresh fruits, fresh vegetables",0.111958,0.606389,0.056565,0.505236,0.833187,-0.011325,0.795552


_Checking for milk, customer prefers milk again with fresh items or dry fruits or nuts_

In [ ]:
custpromo[(custpromo['antecedents'].str.contains('milk')) & (custpromo['confidence']>.50)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
171,milk,fresh fruits,0.485639,0.830598,0.432298,0.890163,1.071713,0.028927,1.542304
173,milk,fresh vegetables,0.485639,0.669402,0.338804,0.697646,1.042193,0.013717,1.093415
178,milk,packaged vegetables fruits,0.485639,0.660023,0.338218,0.696439,1.055174,0.017685,1.119963
182,"fresh vegetables, milk",fresh fruits,0.338804,0.830598,0.316823,0.935121,1.125841,0.035413,2.611051
181,"fresh fruits, milk",fresh vegetables,0.432298,0.669402,0.316823,0.732881,1.094830,0.027442,1.237644
...,...,...,...,...,...,...,...,...,...
27940,"breakfast bakery, milk, yogurt","fresh vegetables, packaged vegetables fruits",0.084408,0.504396,0.050117,0.593750,1.177150,0.007542,1.219948
27938,"breakfast bakery, milk, packaged vegetables fruits","fresh vegetables, yogurt",0.086460,0.374853,0.050117,0.579661,1.546367,0.017708,1.487244
5676,"milk, chips pretzels, bread","fresh vegetables, packaged vegetables fruits",0.088511,0.504396,0.050117,0.566225,1.122580,0.005473,1.142537
6842,"fresh fruits, milk, packaged cheese, soy lactosefree",bread,0.096717,0.348476,0.050117,0.518182,1.486994,0.016414,1.352220


_Same insights of fresh items and refrigerated items hold true_

In [ ]:
custpromo[(custpromo['antecedents'].str.contains('refrigerated')) & (custpromo['confidence']>.50)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23537,refrigerated,fresh fruits,0.308030,0.830598,0.270809,0.879163,1.058470,0.014959,1.401903
23547,refrigerated,fresh vegetables,0.308030,0.669402,0.213658,0.693625,1.036186,0.007461,1.079063
23538,refrigerated,packaged vegetables fruits,0.308030,0.660023,0.213365,0.692674,1.049468,0.010057,1.106239
24680,"fresh vegetables, refrigerated",fresh fruits,0.213658,0.830598,0.196952,0.921811,1.109816,0.019488,2.166564
24679,"fresh fruits, refrigerated",fresh vegetables,0.270809,0.669402,0.196952,0.727273,1.086451,0.015672,1.212192
...,...,...,...,...,...,...,...,...,...
16017,"fresh dips tapenades, refrigerated, packaged vegetables fruits",fresh vegetables,0.061254,0.669402,0.050117,0.818182,1.222258,0.009113,1.818288
16022,"refrigerated, fresh dips tapenades","fresh vegetables, packaged vegetables fruits",0.075615,0.504396,0.050117,0.662791,1.314028,0.011977,1.469721
26526,"fresh fruits, refrigerated, energy granola bars",milk,0.080598,0.485639,0.050117,0.621818,1.280413,0.010976,1.360089
26533,"refrigerated, energy granola bars","fresh fruits, milk",0.091149,0.432298,0.050117,0.549839,1.271899,0.010714,1.261110


_Almost same insights hold true for lunch meat and fresh vegetables or fruits_

In [ ]:
custpromo[(custpromo['antecedents'].str.contains('lunch meat')) & (custpromo['confidence']>.50)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10197,lunch meat,fresh fruits,0.272860,0.830598,0.237691,0.871106,1.048770,0.011053,1.314278
10198,lunch meat,packaged vegetables fruits,0.272860,0.660023,0.207796,0.761547,1.153818,0.027702,1.425758
10201,lunch meat,fresh vegetables,0.272860,0.669402,0.194607,0.713212,1.065446,0.011954,1.152759
10222,"lunch meat, packaged vegetables fruits",fresh fruits,0.207796,0.830598,0.189332,0.911142,1.096972,0.016737,1.906446
10220,"fresh fruits, lunch meat",packaged vegetables fruits,0.237691,0.660023,0.189332,0.796547,1.206847,0.032450,1.671036
...,...,...,...,...,...,...,...,...,...
12731,"fresh fruits, lunch meat, crackers",chips pretzels,0.082943,0.377784,0.050117,0.604240,1.599432,0.018783,1.572206
12721,"lunch meat, crackers","fresh fruits, packaged cheese, packaged vegetables fruits",0.092614,0.329132,0.050117,0.541139,1.644138,0.019635,1.462029
12738,"lunch meat, crackers","fresh fruits, chips pretzels",0.092614,0.333236,0.050117,0.541139,1.623894,0.019255,1.453086
14102,"lunch meat, fresh dips tapenades","fresh vegetables, packaged cheese",0.096424,0.366061,0.050117,0.519757,1.419864,0.014820,1.320038


__Combined level for Customer 4962__

In [ ]:
pnames_4962 = list(data[data['uid']==4962].pname_aisle_department.unique())
len(pnames_4962)

129

In [ ]:
frequent_items_combined = fpgrowth(pname_aisle_department, min_support=0.01, use_colnames=True) #use this by default
frequent_items_combined.head(10) #display 10 results

,support,itemsets
0,0.250293,(Organic Strawberries::fresh fruits::produce)
1,0.248828,(Bag of Organic Bananas::fresh fruits::produce)
2,0.089977,(Organic Avocado::fresh fruits::produce)
3,0.086753,(Organic Large Extra Fancy Fuji Apple::fresh fruits::produce)
4,0.080012,(Organic Whole String Cheese::packaged cheese::dairy eggs)
5,0.051290,(Organic Small Bunch Celery::fresh vegetables::produce)
6,0.042497,(Organic Cilantro::fresh herbs::produce)
7,0.033705,(Organic Red Onion::fresh vegetables::produce)
8,0.030188,(Colby Cheese Sticks::packaged cheese::dairy eggs)
9,0.023740,(Organic Large Brown Grade AA Cage Free Eggs::eggs::dairy eggs)


In [ ]:
#specify suitable parameter values, random values are set below
rules_combined = association_rules(frequent_items_combined, metric="support", min_threshold=0.01)
rules_combined = rules_combined.sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

In [ ]:
rules_combined.shape

(7452, 9)

In [ ]:
##Convert frozenset string to unicode string
rules_combined["antecedents"] = rules_combined["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
rules_combined["consequents"] = rules_combined["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

##Find all items purchased by customer id 4962
rules_combined['LHS'] = rules_combined['antecedents'].str.findall('(' + '|'.join(pnames_4962) + ')')
rules_combined['RHS'] = rules_combined['consequents'].str.findall('(' + '|'.join(pnames_4962) + ')')

##Remove blank rows where the items dont exist
custpromo_combined = rules_combined[rules_combined['LHS'].astype(bool) & rules_combined['RHS'].astype(bool)]
custpromo_combined = custpromo_combined.iloc[:, :9].sort_values(["support", "confidence", "lift", "conviction"], ascending = (False, False, False, False))

custpromo_combined.shape

(306, 9)

In [ ]:
custpromo_combined['lift'].describe()

count    306.000000
mean       2.325428
std        2.968031
min        0.649333
25%        1.620133
50%        1.892518
75%        2.423128
max       37.516204
Name: lift, dtype: float64

In [ ]:
custpromo_combined['conviction'].describe()

count    306.000000
mean       1.237258
std        0.326239
min        0.895930
25%        1.032637
50%        1.114310
75%        1.306565
max        3.311694
Name: conviction, dtype: float64

_Organic products go very well with each other with high confidence and complimentarity_

In [ ]:
custpromo_combined[(custpromo_combined['lift']>2) & (custpromo_combined['confidence']> 0.60)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
100,"Bag of Organic Bananas::fresh fruits::produce, Organic Whole String Cheese::packaged cheese::dairy eggs",Organic Strawberries::fresh fruits::produce,0.034291,0.250293,0.021102,0.615385,2.458656,0.012519,1.949238
68,"Organic Large Extra Fancy Fuji Apple::fresh fruits::produce, Organic Hass Avocado::fresh fruits::produce",Bag of Organic Bananas::fresh fruits::produce,0.027550,0.248828,0.018757,0.680851,2.736235,0.011902,2.353673
5624,"Bag of Organic Bananas::fresh fruits::produce, Honeycrisp Apple::fresh fruits::produce",Organic Strawberries::fresh fruits::produce,0.024326,0.250293,0.016706,0.686747,2.743771,0.010617,2.393295
5650,"Organic Hass Avocado::fresh fruits::produce, Honeycrisp Apple::fresh fruits::produce",Organic Strawberries::fresh fruits::produce,0.024912,0.250293,0.016706,0.670588,2.679212,0.010470,2.275896
5649,"Organic Strawberries::fresh fruits::produce, Honeycrisp Apple::fresh fruits::produce",Organic Hass Avocado::fresh fruits::produce,0.027550,0.213658,0.016706,0.606383,2.838105,0.010820,1.997735
5623,"Organic Strawberries::fresh fruits::produce, Honeycrisp Apple::fresh fruits::produce",Bag of Organic Bananas::fresh fruits::produce,0.027550,0.248828,0.016706,0.606383,2.436960,0.009851,1.908384
124,"Organic Hass Avocado::fresh fruits::produce, Organic Whole String Cheese::packaged cheese::dairy eggs",Organic Strawberries::fresh fruits::produce,0.025791,0.250293,0.015826,0.613636,2.451671,0.009371,1.940418
5630,"Bag of Organic Bananas::fresh fruits::produce, Honeycrisp Apple::fresh fruits::produce",Organic Hass Avocado::fresh fruits::produce,0.024326,0.213658,0.014947,0.614458,2.875899,0.009750,2.039575
4917,"Organic Strawberries::fresh fruits::produce, Organic Hass Avocado::fresh fruits::produce, Organic Raspberries::packaged vegetables fruits::produce",Bag of Organic Bananas::fresh fruits::produce,0.019930,0.248828,0.012016,0.602941,2.423128,0.007057,1.891841
5636,"Bag of Organic Bananas::fresh fruits::produce, Organic Hass Avocado::fresh fruits::produce, Honeycrisp Apple::fresh fruits::produce",Organic Strawberries::fresh fruits::produce,0.014947,0.250293,0.011137,0.745098,2.976902,0.007396,2.941158


_Some organic fruits like cucumber, pear, apples go well with other organic items_

In [ ]:
custpromo_combined[(custpromo_combined['lift']<1.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,Bag of Organic Bananas::fresh fruits::produce,Organic Strawberries::fresh fruits::produce,0.248828,0.250293,0.074443,0.299176,1.195301,0.012163,1.069750
0,Organic Strawberries::fresh fruits::produce,Bag of Organic Bananas::fresh fruits::produce,0.250293,0.248828,0.074443,0.297424,1.195301,0.012163,1.069169
5192,Organic Yellow Onion::fresh vegetables::produce,Organic Strawberries::fresh fruits::produce,0.074736,0.250293,0.025791,0.345098,1.378776,0.007085,1.144762
5193,Organic Strawberries::fresh fruits::produce,Organic Yellow Onion::fresh vegetables::produce,0.250293,0.074736,0.025791,0.103044,1.378776,0.007085,1.031560
5892,Organic Cucumber::fresh vegetables::produce,Bag of Organic Bananas::fresh fruits::produce,0.072392,0.248828,0.023740,0.327935,1.317921,0.005727,1.117708
5230,Organic Garlic::fresh vegetables::produce,Bag of Organic Bananas::fresh fruits::produce,0.072978,0.248828,0.023740,0.325301,1.307335,0.005581,1.113345
5893,Bag of Organic Bananas::fresh fruits::produce,Organic Cucumber::fresh vegetables::produce,0.248828,0.072392,0.023740,0.095406,1.317921,0.005727,1.025442
5231,Bag of Organic Bananas::fresh fruits::produce,Organic Garlic::fresh vegetables::produce,0.248828,0.072978,0.023740,0.095406,1.307335,0.005581,1.024794
4945,Organic Bartlett Pear::fresh fruits::produce,Organic Strawberries::fresh fruits::produce,0.062427,0.250293,0.021981,0.352113,1.406801,0.006356,1.157156
4944,Organic Strawberries::fresh fruits::produce,Organic Bartlett Pear::fresh fruits::produce,0.250293,0.062427,0.021981,0.087822,1.406801,0.006356,1.027840


**Thank You**